In [1]:
# Parameters
notebook_fname = "NN_full_RepeatedKFold_v3_BO_test"


In [2]:
import sys
sys.path.append("../CCA_representation_ML/")

In [3]:
import os
import numpy as np
import pandas as pd

print(f'cpu_count: {os.cpu_count()}')

# Data Path
data_path = '../CCA_representation_ML/01_Dataset_Cleaned/'
if os.path.isfile(data_path+'LiteratureDataset_Corrosion_YW_v3_processed.xlsx'):
    print(f"Folder '{data_path}' found.")
else:
    print(f"Warning: File '{data_path}' not found!")

# Model Path
model_path = '../CCA_representation_ML/04_Model_Saved/'
model_path_bo = f'{model_path}{notebook_fname}/'

# Use GPU or not
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
print('not using GPU')

cpu_count: 40
Folder '../CCA_representation_ML/01_Dataset_Cleaned/' found.
not using GPU


### Check the model and scalers

- Show the model.h5 files in this directory
- Load scalers


In [4]:
import pickle
from utils.postprocessing_evalutation import display_saved_models

display_saved_models(model_path_bo)

# Load the scalers dictionary from a file using pickle
with open(data_path + 'scalers.pkl', 'rb') as f:
    scalers = pickle.load(f)
    print("")
print(scalers)


2023-07-17 19:34:51.641876: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 19:34:54.709406: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/nethome/yuxiang.wu/anaconda3/envs/tf-env/lib/
2023-07-17 19:34:54.709631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/nethome/yuxiang.wu/anaconda3/envs/tf-env/lib/
2023-07-17 19:34:54.709651: W tens

NNH_model                      NNC_model
-----------------------------  -----------------------------
NNH_model_RepeatedKFold_1.h5   NNC_model_RepeatedKFold_1.h5
NNH_model_RepeatedKFold_2.h5   NNC_model_RepeatedKFold_2.h5
NNH_model_RepeatedKFold_3.h5   NNC_model_RepeatedKFold_3.h5
NNH_model_RepeatedKFold_4.h5   NNC_model_RepeatedKFold_4.h5
NNH_model_RepeatedKFold_5.h5   NNC_model_RepeatedKFold_5.h5
NNH_model_RepeatedKFold_6.h5   NNC_model_RepeatedKFold_6.h5
NNH_model_RepeatedKFold_7.h5   NNC_model_RepeatedKFold_7.h5
NNH_model_RepeatedKFold_8.h5   NNC_model_RepeatedKFold_8.h5
NNH_model_RepeatedKFold_9.h5   NNC_model_RepeatedKFold_9.h5
NNH_model_RepeatedKFold_10.h5  NNC_model_RepeatedKFold_10.h5
NNH_model_RepeatedKFold_11.h5  NNC_model_RepeatedKFold_11.h5
NNH_model_RepeatedKFold_12.h5  NNC_model_RepeatedKFold_12.h5

{'compo': MinMaxScaler(), 'H_specific_testing': MinMaxScaler(), 'C_specific_testing': MinMaxScaler(), 'specific_features': MinMaxScaler(), 'H_output': MinMaxScaler(), 'C_outp

## Predict based on new data inputs


### coordinates for PVD alloy representation


In [5]:
# load the wafer-coordinates
df_PVD_x_y = pd.read_excel(data_path + 'PVD_x_y.xlsx')
coord_x = df_PVD_x_y["x"].to_numpy(dtype=float)
coord_y = df_PVD_x_y["y"].to_numpy(dtype=float)
index_PVD_x_y = df_PVD_x_y.index.values+1


### Setting up compositional input for new alloys


In [6]:
from itertools import combinations

Flag_Calc_KW = False
Flag_Calc_All = True

if Flag_Calc_KW:
    data_path_compo = './v6_A-B-C-D-E_Sputtering_ML_Exp/'

    compo_A_B_C_D_E_list = [['Ni', 'Cr', 'Co', 'V', 'Fe'],
                            ['Ni', 'Cr', 'Mo', 'Ti', 'Fe']]
    compo_A_B_C_D_E_fname_list = ['KW99', 'KW131']


elif Flag_Calc_All:
    data_path_compo = './v6_A-B-C-D-E_Sputtering_ML_All/'

    # Define the set of elements for D and E
    set_D_E = [{'Co'}, {'V'}, {'Mn'}, {'Mo'}, {'Cu'},
               {'Nb'}, {'W'}, {'Ti'}, {'Al'}, {'Si'}, {'Ta'}]

    # Generate all possible combinations of D and E
    compo_D_E_list = list(combinations(set_D_E, 2))

    compo_A_B_C_D_E_list = []
    compo_A_B_C_D_E_fname_list = []
    for compo_D_E in compo_D_E_list:
        # Convert the current combination to a list and flatten it
        compo_D_E_list = [item for sublist in compo_D_E for item in sublist]

        # Define the current combination of A, B, C, D, E
        compo_A_B_C_D_E = ['Fe', 'Cr', 'Ni'] + compo_D_E_list
        compo_A_B_C_D_E_merge = '_'.join([*compo_A_B_C_D_E])

        compo_A_B_C_D_E_list.append(compo_A_B_C_D_E)
        compo_A_B_C_D_E_fname_list.append(compo_A_B_C_D_E_merge)

    print(compo_A_B_C_D_E_list[0])
    print(compo_A_B_C_D_E_fname_list[0])


['Fe', 'Cr', 'Ni', 'Co', 'V']
Fe_Cr_Ni_Co_V


### Make predictions for new alloys


In [7]:
from utils.postprocessing_prediction import read_new_data_feature_calc, predict_bootstrap_NNH_NNC, plot_prediction_uncertainty, plot_prediction_uncertainty_AVG
from tqdm import tqdm

for compo_A_B_C_D_E, compo_A_B_C_D_E_fname in tqdm(zip(compo_A_B_C_D_E_list, compo_A_B_C_D_E_fname_list), desc='Processing', total=len(compo_A_B_C_D_E_list)):

    print(compo_A_B_C_D_E)
    print(compo_A_B_C_D_E_fname)
    vars_ele, KW_name = compo_A_B_C_D_E, compo_A_B_C_D_E_fname

    file_name_input = f'{data_path_compo}v6_{KW_name}_SSS_FCC_byCompo_wt_pct.xlsx'
    df_new_wt = pd.read_excel(file_name_input)

    compo_new, HC_specific_features, C_specific_testing = read_new_data_feature_calc(df_new_wt, vars_ele,
                                                                                     specific_features_sel_column=['delta_a', 'Tm', 'sigma_Tm',
                                                                                                                   'Hmix', 'sigma_Hmix', 'sigma_elec_nega', 'VEC', 'sigma_VEC'],
                                                                                     C_testing=np.array([25, 1, 7, 0.333]))

    NNH_model_name = 'NNH_model_RepeatedKFold_{}.h5'
    NNC_model_name = 'NNC_model_RepeatedKFold_{}.h5'
    k_folds, n_CVrepeats, mc_repeat = 6, 2, 100

    (H1_new_pred_stack, H1_new_pred_mean, H1_new_pred_std,
     C2_new_pred_stack, C2_new_pred_mean, C2_new_pred_std) = predict_bootstrap_NNH_NNC(
        model_path_bo, NNH_model_name, NNC_model_name,
        compo_new, HC_specific_features, C_specific_testing,
        scalers, k_folds, n_CVrepeats, mc_repeat)

    # Concatenate and compute mean and std + save to excel
    H1_new_pred_KFold_mean = np.mean(np.concatenate(
        H1_new_pred_stack, axis=0), axis=0).reshape(-1)
    H1_new_pred_KFold_std = np.std(np.concatenate(
        H1_new_pred_stack, axis=0), axis=0).reshape(-1)
    C2_new_pred_KFold_mean = np.mean(np.concatenate(
        C2_new_pred_stack, axis=0), axis=0).reshape(-1)
    C2_new_pred_KFold_std = np.std(np.concatenate(
        C2_new_pred_stack, axis=0), axis=0).reshape(-1)

    df_new_wt['H1_new_pred_KFold_mean'] = H1_new_pred_KFold_mean
    df_new_wt['H1_new_pred_KFold_std'] = H1_new_pred_KFold_std
    df_new_wt['C2_new_pred_KFold_mean'] = C2_new_pred_KFold_mean
    df_new_wt['C2_new_pred_KFold_std'] = C2_new_pred_KFold_std

    file_name_output = f'{data_path_compo}v6_{KW_name}_SSS_FCC_byCompo_wt_pct_ML.xlsx'
    display(df_new_wt.head(3))
    df_new_wt.to_excel(file_name_output, index=False)

    if Flag_Calc_KW:
        # NNH predictions
        plot_prediction_uncertainty(data_path_compo, coord_x, coord_y, index_PVD_x_y, H1_new_pred_mean, H1_new_pred_std,
                                    pred_label='Hardness', unc_label='Hardness uncertainty',
                                    title='NNH_RepeatedKFold_prediction_uncertainty_eachFold_' + KW_name,
                                    vmin1=100, vmax1=300,
                                    vmin2=25, vmax2=100)

        # NNC predictions
        plot_prediction_uncertainty(data_path_compo, coord_x, coord_y, index_PVD_x_y, C2_new_pred_mean, C2_new_pred_std,
                                    pred_label='Pitting potential (mV)', unc_label='Pitting potential uncertainty (mV)',
                                    title='NNC_RepeatedKFold_prediction_uncertainty_eachFold_' + KW_name,
                                    vmin1=0, vmax1=900,
                                    vmin2=50, vmax2=150)

        # NNH_NNC_AVG predictions
        plot_prediction_uncertainty_AVG(data_path_compo, coord_x, coord_y, index_PVD_x_y, H1_new_pred_stack, C2_new_pred_stack,
                                        title='NNH_NNC_RepeatedKFold_prediction_uncertainty_AVG_' + KW_name)


Processing:   0%|          | 0/55 [00:00<?, ?it/s]

['Fe', 'Cr', 'Ni', 'Co', 'V']
Fe_Cr_Ni_Co_V


2023-07-17 19:35:03.975748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 19:35:03.984450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 19:35:03.985164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

,Unnamed: 0,Fe,Cr,Ni,Co,V,ShearModulus_eff,PoissonsRatio_eff,delta_prime_misfit,sigma_SSS,Gmin_FCC,eq_FCC,Gmin_eq_FCC,sigma_SSS_FCC,compo_A_B_C_D_E,H1_new_pred_KFold_mean,H1_new_pred_KFold_std,C2_new_pred_KFold_mean,C2_new_pred_KFold_std
0,0,34.167395,2.891940,5.441344,13.111107,44.388214,63.740744,0.329844,0.092273,456.766541,0,0.0,0,NaN,Fe-Cr-Ni-Co-V,378.698212,173.712723,200.080307,347.795746
1,1,40.204113,4.798947,5.417690,11.966268,37.612981,66.837689,0.322980,0.091240,464.791066,0,0.0,0,NaN,Fe-Cr-Ni-Co-V,413.697540,166.767456,255.091766,289.118958
2,2,45.696355,6.768571,4.366436,10.959457,32.209181,69.598860,0.316791,0.088701,457.512234,0,0.0,0,NaN,Fe-Cr-Ni-Co-V,421.226593,156.832321,273.563660,251.485764


Processing:   2%|▏         | 1/55 [01:16<1:08:26, 76.05s/it]

...

Processing:  98%|█████████▊| 54/55 [1:07:34<01:15, 75.42s/it]

['Fe', 'Cr', 'Ni', 'Si', 'Ta']
Fe_Cr_Ni_Si_Ta


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '__dict__'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '__dict__'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '__dict__'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosi

,Unnamed: 0,Fe,Cr,Ni,Si,Ta,ShearModulus_eff,PoissonsRatio_eff,delta_prime_misfit,sigma_SSS,Gmin_FCC,eq_FCC,Gmin_eq_FCC,sigma_SSS_FCC,compo_A_B_C_D_E,H1_new_pred_KFold_mean,H1_new_pred_KFold_std,C2_new_pred_KFold_mean,C2_new_pred_KFold_std
0,0,16.552437,1.401004,2.636066,3.026994,76.383500,64.966643,0.335322,0.154332,1095.396810,0,0.0,0,NaN,Fe-Cr-Ni-Si-Ta,756.266541,155.183365,826.397400,568.506836
1,1,21.190452,2.529390,2.855511,3.005735,70.418912,66.750948,0.329984,0.159378,1157.692614,0,0.0,0,NaN,Fe-Cr-Ni-Si-Ta,738.099548,149.333633,812.735474,556.726318
2,2,25.895626,3.835676,2.474412,2.959757,64.834529,68.529438,0.324783,0.160516,1179.354805,0,0.0,0,NaN,Fe-Cr-Ni-Si-Ta,722.618469,136.211227,818.898926,542.828430


Processing: 100%|██████████| 55/55 [1:08:52<00:00, 75.14s/it]
